# Preprocessing the Rossmann Store Sales Dataset
Here we implement some feature engineering outlined by FastAI in [their example solution](https://github.com/fastai/fastai/blob/master/courses/dl1/lesson3-rossman.ipynb) to the [Kaggle Rossmann Store Sales competition](https://www.kaggle.com/c/rossmann-store-sales). We've simplified some sections and left out most of the documentation to keep things neat, so feel free to consult the original notebook for explanations of the feature engineering going on.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
INPUT_DATA_DIR = os.environ.get('INPUT_DATA_DIR', '/tmp/rossmann')
OUTPUT_DATA_DIR = os.environ.get('DATA_DIR', '/data')
VALID_FRAC = 0.25

In [3]:
! mkdir -p $INPUT_DATA_DIR
! wget -O $INPUT_DATA_DIR/rossmann.tgz http://files.fast.ai/part2/lesson14/rossmann.tgz
! cd $INPUT_DATA_DIR && tar -xzf rossmann.tgz && ls

--2020-05-11 22:13:57--  http://files.fast.ai/part2/lesson14/rossmann.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7730448 (7.4M) [application/x-gtar-compressed]
Saving to: ‘/tmp/rossmann/rossmann.tgz’

/tmp/rossmann/rossm 100%[===================>]   7.37M  8.60MB/s    in 0.9s    

2020-05-11 22:13:58 (8.60 MB/s) - ‘/tmp/rossmann/rossmann.tgz’ saved [7730448/7730448]

googletrend.csv        state_names.csv	 test.csv
rossmann.tgz	       store.csv	 train.csv
sample_submission.csv  store_states.csv  weather.csv


In [4]:
def read_table(table_name):
    return pd.read_csv(os.path.join(INPUT_DATA_DIR, f'{table_name}.csv'), low_memory=False)

train = read_table('train')
store = read_table('store')
store_states = read_table('store_states')
state_names = read_table('state_names')
googletrend = read_table('googletrend')
weather = read_table('weather')
test = read_table('test')

In [5]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

In [6]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

trend_de = googletrend.loc[googletrend.file == 'Rossmann_DE'].copy()

In [7]:
for df in (weather, googletrend, train, test, trend_de):
    df.loc[:, 'Date'] = pd.to_datetime(df.Date)
    df['Year'] = df.Date.dt.year
    df['Month'] = df.Date.dt.month
    df['Week'] = df.Date.dt.week
    df['Day'] = df.Date.dt.day

In [8]:
# quick extension for handling left merges succinctly
@pd.api.extensions.register_dataframe_accessor('left')
class LeftMerger:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    def merge(self, right, left_on, right_on=None, suffix=None):
        df = self._obj.merge(
            right, how='left', left_on=left_on, right_on=right_on or left_on, suffixes=('', suffix or '_y'))
        if suffix is None:
            return df.drop(columns=df.filter(regex='_y$').columns.tolist())
        return df

In [9]:
weather = weather.left.merge(state_names, 'file', right_on='StateName')
store = store.left.merge(store_states, 'Store', right_on='Store')

In [10]:
train_df = train.left.merge(store, 'Store')
test_df = test.left.merge(store, 'Store')

In [11]:
train_df = train_df.left.merge(googletrend, ['State', 'Year', 'Week'])
test_df = test_df.left.merge(googletrend, ['State', 'Year', 'Week'])

In [12]:
train_df = train_df.left.merge(trend_de, ['Year', 'Week'], suffix='_DE')
test_df = test_df.left.merge(trend_de, ['Year', 'Week'], suffix='_DE')

In [13]:
train_df = train_df.left.merge(weather, ['State', 'Date'])
test_df = test_df.left.merge(weather, ['State', 'Date'])

In [14]:
for df in [train_df, test_df]:
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [15]:
for df in [train_df, test_df]:
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [16]:
for df in [train_df, test_df]:
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [17]:
for df in [train_df, test_df]:
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24

In [18]:
for df in [train_df, test_df]:
    dt = pd.to_datetime(df.Promo2SinceYear, format='%Y').astype(np.int64) // 10**9
    dt += 7*24*3600*df.Promo2SinceWeek
    df["Promo2Since"] = pd.to_datetime(dt*10**9)
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [19]:
for df in [train_df, test_df]:
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [20]:
df = train_df.append(test_df, ignore_index=True)

/conda/envs/rapids/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [21]:
# ops: masking, ffill, bfill, timedelta
df = df.sort_values(by=['Store', 'Date'])

# first build a mask indicating where stores start and end
first_indices = df.Store.diff() != 0
last_indices = df.Store.diff().iloc[1:].append(pd.Series([1]))
last_indices.index = first_indices.index
idx_mask = ~(first_indices | last_indices)

event_fields = ['SchoolHoliday', 'StateHoliday', 'Promo']
for field in event_fields:
    # use the mask from above to mask save dates from the start and end
    # of a given store's range, as well as all dates that have an event
    df['tmp'] = df.Date
    df.loc[(df[field] == 0) & idx_mask, 'tmp'] = np.nan

    # then use ffill and bbfill to give the input to the time delta
    df['After'+field] = df.tmp.ffill()
    df['Before'+field] = df.tmp.bfill()

    # compute deltas between bfilled and ffilled dates and the current date
    df['After'+field] = (df['Date'] - df['After'+field]).astype('timedelta64[D]')
    df['Before'+field] = (df['Before'+field] - df['Date']).astype('timedelta64[D]')

# get rid of our dummy column
df = df.drop(columns=['tmp'])

In [22]:
df = df.set_index("Date")
bwd = df[['Store']+event_fields].sort_index().groupby("Store").rolling(7, min_periods=1).sum()
fwd = df[['Store']+event_fields].sort_index(ascending=False).groupby("Store").rolling(7, min_periods=1).sum()

In [23]:
for d in (bwd, fwd):
    d.drop('Store', 1, inplace=True)
    d.reset_index(inplace=True)

In [24]:
df.reset_index(inplace=True)

In [25]:
for d, suffix in zip([bwd, fwd], ['_bw', '_fw']):
    df = df.left.merge(d, ['Store', 'Date'], suffix=suffix)

In [26]:
train_df = train_df.left.merge(df, ['Store', 'Date'])
test_df = test_df.left.merge(df, ['Store', 'Date'])

In [27]:
train_df = train_df[train_df.Sales != 0]

In [28]:
train_df = train_df.sort_values(by='Date')
num_valid = int(VALID_FRAC*len(train_df))
valid_df = train_df[-num_valid:]
train_df = train_df[:-num_valid]

In [29]:
train_df.to_csv(os.path.join(OUTPUT_DATA_DIR, 'train.csv'), index=False)
valid_df.to_csv(os.path.join(OUTPUT_DATA_DIR, 'valid.csv'), index=False)
test_df.to_csv(os.path.join(OUTPUT_DATA_DIR, 'test.csv'), index=False)